In [9]:
import sys
sys.path.append("..")
import torch
from torch import nn
from torch import optim
import torchvision.transforms as transforms
import time
import os
from Tensorized_components.patch_embedding  import Patch_Embedding     
from Tensorized_components.w_msa_w_o_b_sign  import WindowMSA     
from Tensorized_components.sh_wmsa_w_o_b_sign import ShiftedWindowMSA     
from Tensorized_components.patch_merging  import TensorizedPatchMerging  
from Tensorized_Layers.TCL_CHANGED import TCL_CHANGED   
from Tensorized_Layers.TRL import TRL   
from Utils.Accuracy_measures import topk_accuracy
from Utils.TinyImageNet_loader import get_tinyimagenet_dataloaders
from Utils.Num_parameter import count_parameters




# ----------------------- imports & basic set-up -----------------------
import os, time, math, torch, torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast


In [10]:
class SwinBlock1(nn.Module):
    """
    A class representing 'Block 1' in your Swin Transformer.
    This captures the sequence of:
        (1) Window MSA + residual
        (2) TCL + residual
        (3) Shifted Window MSA + residual
        (4) TCL + residual
    but only for the first block’s hyperparameters and submodules.
    """
    def __init__(self, w_msa, sw_msa, tcl, embed_shape, dropout=0):
        super(SwinBlock1, self).__init__()
        # Typically each sub-layer has its own LayerNorm
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # We pass in pre-built modules (WindowMSA, ShiftedWindowMSA, TCL)
        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl = tcl

    def forward(self, x):
        # ----- First Window MSA + Residual -----
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        # ----- TCL + Residual -----
        x_res = x
        x = self.norm2(x)
        x = self.tcl(x)
        x = x + x_res

        # ----- Shifted Window MSA + Residual -----
        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        # ----- TCL + Residual -----
        x_res = x
        x = self.norm4(x)
        x = self.tcl(x)
        x = x + x_res

        return x


In [11]:
class SwinBlock2(nn.Module):
    def __init__(self, w_msa, sw_msa, tcl, embed_shape=(4,4,6), dropout=0):
        super(SwinBlock2, self).__init__()
        # LN layers
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        self.dropout = nn.Dropout(dropout)

        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl = tcl

    def forward(self, x):
        # Window MSA
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        # TCL
        x_res = x
        x = self.norm2(x)
        x = self.tcl(x)
        x = x + x_res

        # Shifted Window MSA
        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        # TCL
        x_res = x
        x = self.norm4(x)
        x = self.tcl(x)
        x = x + x_res

        return x


In [12]:
class SwinBlock3(nn.Module):
    def __init__(self, w_msa, sw_msa, tcl, embed_shape=(4,4,12), dropout=0):
        super(SwinBlock3, self).__init__()
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        self.dropout = nn.Dropout(dropout)

        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl = tcl

    def forward(self, x):
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        x_res = x
        x = self.norm2(x)
        x = self.tcl(x)
        x = x + x_res

        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        x_res = x
        x = self.norm4(x)
        x = self.tcl(x)
        x = x + x_res
        return x

In [13]:
class SwinBlock4(nn.Module):
    def __init__(self, w_msa, sw_msa, tcl, embed_shape=(4,4,24), dropout=0):
        super(SwinBlock4, self).__init__()
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        self.dropout = nn.Dropout(dropout)
        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl = tcl

    def forward(self, x):
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        x_res = x
        x = self.norm2(x)
        x = self.tcl(x)
        x = x + x_res

        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        x_res = x
        x = self.norm4(x)
        x = self.tcl(x)
        x = x + x_res

        return x


In [14]:
class SwinTransformer(nn.Module):
    def __init__(self,
                 img_size=224,
                 patch_size=4,
                 in_chans=3,
                 embed_shape=(4,4,12),
                 bias=True,
                 dropout=0,
                 device="cuda"):
        super(SwinTransformer, self).__init__()

        self.device = device


        self.patch_embedding = Patch_Embedding(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_shape=embed_shape,
            bias=bias
        )

        # -------------------------------- block 1 --------------------------

        self.w_msa_1 = WindowMSA(
            window_size=7,
            embed_dims=embed_shape,
            rank_window=embed_shape,
            head_factors=(1,2,3),
            device=self.device
        )

        self.sw_msa_1 = ShiftedWindowMSA(
            window_size=7,
            embed_dims=embed_shape,
            rank_window=embed_shape,
            head_factors=(1,2,3),
            device=self.device
        )

        self.tcl_1 = TCL_CHANGED(
            input_size=(16, 56, 56, 4,4,12),
            rank=(4,4,12),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        self.block1_list = nn.ModuleList([
            SwinBlock1(
                w_msa=self.w_msa_1,
                sw_msa=self.sw_msa_1,
                tcl=self.tcl_1,
                embed_shape=embed_shape,
                dropout=dropout
            )
            for _ in range(2)
        ])

        # -------------------------------- block 2 --------------------------


        self.patch_merging_1 = TensorizedPatchMerging(
            input_size=(16, 56, 56, 4,4,12),
            in_embed_shape=embed_shape,
            out_embed_shape=(4,4,24),
            bias=bias,
            ignore_modes=(0, 1, 2),
            device=self.device
        )

        self.w_msa_2 = WindowMSA(
            window_size=7,
            embed_dims=(4,4,24),
            rank_window=(4,4,24),
            head_factors=(1,2,6),
            device=self.device
        )

        self.sw_msa_2 = ShiftedWindowMSA(
            window_size=7,
            embed_dims=(4,4,24),
            rank_window=(4,4,24),
            head_factors=(1,2,6),
            device=self.device
        )

        self.tcl_2 = TCL_CHANGED(
            input_size=(16, 28, 28, 4,4,24),
            rank=(4,4,24),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        # We repeat Block2 two times
        self.block2_list = nn.ModuleList([
            SwinBlock2(
                w_msa=self.w_msa_2,
                sw_msa=self.sw_msa_2,
                tcl=self.tcl_2,
                embed_shape=(4,4,24),  
                dropout=dropout
            )
            for _ in range(2)
        ])


        # # -------------------------------- block 3 --------------------------

        self.patch_merging_2 = TensorizedPatchMerging(
            input_size=(16, 28, 28, 4,4,24),
            in_embed_shape=(4,4,24),
            out_embed_shape=(4,4,48),
            bias=bias,
            ignore_modes=(0, 1, 2),
            device=self.device
        )


        self.w_msa_3 = WindowMSA(
            window_size=7,
            embed_dims=(4,4,48),
            rank_window=(4,4,48),
            head_factors=(2,1,12),
            device=self.device
        )

        self.sw_msa_3 = ShiftedWindowMSA(
            window_size=7,
            embed_dims=(4,4,48),
            rank_window=(4,4,48),
            head_factors=(2,1,12),
            device=self.device
        )

        self.tcl_3 = TCL_CHANGED(
            input_size=(16, 14, 14, 4,4,48),
            rank=(4,4,48),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        # Repeat Block3 6 times
        self.block3_list = nn.ModuleList([
            SwinBlock3(
                w_msa=self.w_msa_3,
                sw_msa=self.sw_msa_3,
                tcl=self.tcl_3,
                embed_shape=(4,4,48),
                dropout=dropout
            )
            for _ in range(18)
        ])

        # # # -------------------------------- block 4 --------------------------

        self.patch_merging_3 = TensorizedPatchMerging(
            input_size=(16, 14, 14, 4,4,48),
            in_embed_shape=(4,4,48),
            out_embed_shape=(4,4,96),
            bias=bias,
            ignore_modes=(0, 1, 2),
            device=self.device
        )

        self.w_msa_4 = WindowMSA(
            window_size=7,
            embed_dims=(4,4,96),
            rank_window=(4,4,96),
            head_factors=(2,1,24),
            device=self.device
        )

        self.sw_msa_4 = ShiftedWindowMSA(
            window_size=7,
            embed_dims=(4,4,96),
            rank_window=(4,4,96),
            head_factors=(2,1,24),
            device=self.device
        )

        self.tcl_4 = TCL_CHANGED(
            input_size=(16, 7, 7, 4,4,96),
            rank=(4,4,96),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )


        self.block4_list = nn.ModuleList([
            SwinBlock4(
                w_msa=self.w_msa_4,
                sw_msa=self.sw_msa_4,
                tcl=self.tcl_4,
                embed_shape=(4,4,96),
                dropout=dropout
            )
            for _ in range(2)
        ])

        # -------------------------------- classifier --------------------------

    

        self.classifier = TRL(input_size=(16,4,4,96),
                            output=(200,),
                            rank=(4,4,96,200),
                            ignore_modes=(0,),
                            bias=bias,
                            device=self.device) 
        

        # positoin embedding


        self.pos_embedding = nn.Parameter(
            torch.randn(1,
                        56,
                        56,
                        4,
                        4,
                        12,
                        device = self.device
                        ), requires_grad=True)

    def forward(self, x):
 

        x = self.patch_embedding(x)

        x += self.pos_embedding

        for i, blk in enumerate(self.block1_list, 1):
            x = blk(x)


        x = self.patch_merging_1(x)



        for i, blk in enumerate(self.block2_list, 1):
            x = blk(x)


        x = self.patch_merging_2(x)

        for i, blk in enumerate(self.block3_list, 1):
            x = blk(x)


        x = self.patch_merging_3(x)


        for i, blk in enumerate(self.block4_list, 1):
            x = blk(x)


        x = x.mean(dim=(1, 2))

        output = self.classifier(x)
        return output

In [15]:
# Setup the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f'Device is set to : {device}')

# Configs

TEST_ID = 'Test_ID015'
batch_size = 16
n_epoch = 400
image_size = 224

model = SwinTransformer(img_size=224,patch_size=4,in_chans=3,embed_shape=(4,4,12),bias=True,device=device).to(device)


# Set up the transforms and train/test loaders

tiny_transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((image_size, image_size)), 
        transforms.RandomCrop(image_size, padding=5),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_val = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_test = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


train_loader, val_loader , test_loader = get_tinyimagenet_dataloaders(
                                                    data_dir = '../datasets',
                                                    transform_train=tiny_transform_train,
                                                    transform_val=tiny_transform_val,
                                                    transform_test=tiny_transform_test,
                                                    batch_size=batch_size,
                                                    image_size=image_size)

Device is set to : cpu


In [16]:


num_parameters = count_parameters(model)
print(f'This model has {num_parameters:,} parameters')


# ----------------------- loss, optimizer, scheduler ------------------
criterion       = nn.CrossEntropyLoss()

base_lr         = 5e-4          # “large-batch” LR from Swin paper
weight_decay    = 0.05
betas           = (0.9, 0.999)

optimizer       = optim.AdamW(model.parameters(),
                              lr=base_lr,
                              betas=betas,
                              weight_decay=weight_decay)

warmup_epochs   = 5
scheduler       = CosineAnnealingLR(optimizer, T_max=n_epoch - warmup_epochs)

# ------------- gradient accumulation to fake batch-size 512 ----------
effective_bs    = 512
accum_steps     = math.ceil(effective_bs / batch_size)   # 512/16 = 32
print(f'Accumulating gradients for {accum_steps} steps '
      f'→ effective batch-size {effective_bs}')

scaler = GradScaler()   # for mixed precision

# ----------------------- directories for results ---------------------
result_dir   = os.path.join('../results', TEST_ID)
acc_dir      = os.path.join(result_dir, 'accuracy_stats')
model_dir    = os.path.join(result_dir, 'model_stats')
os.makedirs(acc_dir,   exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
with open(os.path.join(model_dir, 'model_info.txt'), 'a') as f:
    f.write(f'Total parameters: {num_parameters:,}\n')

# ----------------------- train & test epochs -------------------------
def train_epoch(loader, epoch):
    model.train()
    start_time     = time.time()
    running_loss   = 0.0
    correct        = {k: 0.0 for k in (1,2,3,4,5)}
    step_in_accum  = 0

    # warm-up or cosine step
    if epoch <= warmup_epochs:
        warm_lr = base_lr * epoch / warmup_epochs
        for g in optimizer.param_groups: g['lr'] = warm_lr
    else:
        scheduler.step()

    optimizer.zero_grad(set_to_none=True)

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        with autocast():
            outputs = model(inputs)
            loss    = criterion(outputs, targets) / accum_steps  # scale for accum

        scaler.scale(loss).backward()
        step_in_accum += 1

        if step_in_accum == accum_steps:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            step_in_accum = 0   # reset counter

        running_loss += loss.item() * accum_steps  # undo scaling for log
        accs = topk_accuracy(outputs, targets, topk=(1,2,3,4,5))
        for k in accs: correct[k] += accs[k]['correct']

    elapsed = time.time() - start_time
    top_acc = [correct[k]/len(loader.dataset) for k in correct]
    avg_loss = running_loss / len(loader.dataset)

    report = (f'Train epoch {epoch:3d}: '
              f'top1={top_acc[0]:.4%}, top2={top_acc[1]:.4%}, '
              f'top3={top_acc[2]:.4%}, top4={top_acc[3]:.4%}, '
              f'top5={top_acc[4]:.4%}, loss={avg_loss:.4f}, '
              f'time={elapsed:.1f}s')
    print(report)
    return report


@torch.no_grad()
def test_epoch(loader, epoch, tag='Test'):
    model.eval()
    start_time   = time.time()
    running_loss = 0.0
    correct      = {k: 0.0 for k in (1,2,3,4,5)}

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        with autocast():
            outputs = model(inputs)
            loss    = criterion(outputs, targets)

        running_loss += loss.item()
        accs = topk_accuracy(outputs, targets, topk=(1,2,3,4,5))
        for k in accs: correct[k] += accs[k]['correct']

    elapsed = time.time() - start_time
    top_acc = [correct[k]/len(loader.dataset) for k in correct]
    avg_loss = running_loss / len(loader.dataset)

    report = (f'{tag} epoch {epoch:3d}: '
              f'top1={top_acc[0]:.4%}, top2={top_acc[1]:.4%}, '
              f'top3={top_acc[2]:.4%}, top4={top_acc[3]:.4%}, '
              f'top5={top_acc[4]:.4%}, loss={avg_loss:.4f}, '
              f'time={elapsed:.1f}s')
    print(report)
    return report

# ----------------------- main training loop --------------------------
print(f'\nTraining for {n_epoch} epochs …\n')
for epoch in range(1, n_epoch + 1):
    report_tr = train_epoch(train_loader, epoch)
    # uncomment if you want validation/testing each epoch:
    # report_va = test_epoch(val_loader,  epoch, tag='Val')

    # -------- persist logs & checkpoints ----------
    with open(os.path.join(acc_dir, 'report_train.txt'), 'a') as f:
        f.write(report_tr + '\n')

    if epoch % 5 == 0:
        torch.save(model.state_dict(),
                   os.path.join(model_dir, f'Model_epoch_{epoch}.pth'))

This model has 1,500,184 parameters
Accumulating gradients for 32 steps → effective batch-size 512

Training for 400 epochs …



C:\Users\m.badzohreh\AppData\Local\Temp\ipykernel_5344\397044912.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()   # for mixed precision
c:\Users\m.badzohreh\AppData\Local\anaconda3\envs\data_science\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
C:\Users\m.badzohreh\AppData\Local\Temp\ipykernel_5344\397044912.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
c:\Users\m.badzohreh\AppData\Local\anaconda3\envs\data_science\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 